In [1]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import torch.optim as optim
import numpy as np
import plotly.graph_objs as go
import pandas
import os
import psutil #to get the memory used
import os


In [2]:
if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

print(device)

cuda


In [3]:
combined_sequences = []

leave = []  # List to store indices of files not considered
####change foolder location
dirs = os.listdir('NMP')
c = 0
for d in dirs:
    count = len(list(os.listdir(os.path.join('NMP', d))))
    print(count)
    for i in range(count):
        c += 1
        # Load the .npz file
        ####change foolder location before NMP
        data = np.load(f'NMP/{d}/{d}_{i}.npz')
        
        # Extract relevant data
        position_data = data['position']  # Assuming 'position' is a key in the .npz file
        
        # Extract x, y, z coordinates
        x = position_data[:, 0][:100]  # Get the first 100 x coordinates
        y = position_data[:, 1][:100]  # Get the first 100 y coordinates
        z = position_data[:, 2][:100]  # Get the first 100 z coordinates
        
        # Check if the sequence is long enough
        if len(x) == 100:
            combined_sequence = np.vstack((x, y, z)).T  # Stack x, y, z and transpose to get shape (100, 3)
            combined_sequences.append(combined_sequence)
             ####change foolder location before NMP
            print(f'NMP/trimmed_Bamboo/trimmed_Bamboo_{i}.npz - Loaded successfully')
        else:
             ####change foolder location before NMP
            leave.append(f'NMP/{d}/{d}_{i}.npz')
            print(f'NMP/trimmed_Bamboo/trimmed_Bamboo_{i}.npz - left')
        
        print("---------------------------")
        #break

    # Convert list to NumPy array
    combined_array = np.array(combined_sequences)

    # Convert NumPy array to PyTorch tensor
    combined_tensor = torch.tensor(combined_array, dtype=torch.float32)


    # Now combined_tensor can be used for training your LSTM model
print(combined_tensor.shape)  # Should print torch.Size([163, 100, 3])

print(c)

168
NMP/trimmed_Bamboo/trimmed_Bamboo_0.npz - Loaded successfully
---------------------------
NMP/trimmed_Bamboo/trimmed_Bamboo_1.npz - Loaded successfully
---------------------------
NMP/trimmed_Bamboo/trimmed_Bamboo_2.npz - Loaded successfully
---------------------------
NMP/trimmed_Bamboo/trimmed_Bamboo_3.npz - Loaded successfully
---------------------------
NMP/trimmed_Bamboo/trimmed_Bamboo_4.npz - Loaded successfully
---------------------------
NMP/trimmed_Bamboo/trimmed_Bamboo_5.npz - Loaded successfully
---------------------------
NMP/trimmed_Bamboo/trimmed_Bamboo_6.npz - Loaded successfully
---------------------------
NMP/trimmed_Bamboo/trimmed_Bamboo_7.npz - Loaded successfully
---------------------------
NMP/trimmed_Bamboo/trimmed_Bamboo_8.npz - Loaded successfully
---------------------------
NMP/trimmed_Bamboo/trimmed_Bamboo_9.npz - Loaded successfully
---------------------------
NMP/trimmed_Bamboo/trimmed_Bamboo_10.npz - Loaded successfully
---------------------------
NMP/t

In [4]:
combined_tensor = combined_tensor.to(device)

In [5]:

test_data = np.load(f'NMP/trimmed_Bamboo/trimmed_Bamboo_{47}.npz')

position_data = test_data['position'][:100]  # Limit to the first 100 samples if necessary

mid_point = position_data.shape[0] // 2

# Prepare the input and target tensors
test_input = torch.tensor(position_data[:mid_point], dtype=torch.float32).unsqueeze(0)  # First half
test_target = torch.tensor(position_data[mid_point:], dtype=torch.float32).unsqueeze(0)  # Second half

# Transfer tensors to the appropriate device (GPU or CPU)
test_input_4 = test_input.to(device)
test_target_4 = test_target.to(device)


In [6]:
position_data = torch.tensor(position_data) #this is combined version of test
position_data = position_data.to(device)

In [7]:
combined_tensor

tensor([[[ 1.0006,  1.2619,  1.8557],
         [ 0.9993,  1.3053,  1.8195],
         [ 0.9999,  1.3470,  1.7853],
         ...,
         [ 0.8238,  2.0998, -1.2494],
         [ 0.8214,  2.0753, -1.2787],
         [ 0.8192,  2.0507, -1.3079]],

        [[ 1.0979,  1.2421,  1.8938],
         [ 1.0934,  1.2864,  1.8607],
         [ 1.0895,  1.3295,  1.8273],
         ...,
         [ 0.8766,  2.2128, -1.1789],
         [ 0.8738,  2.1914, -1.2095],
         [ 0.8710,  2.1696, -1.2403]],

        [[ 0.9545,  0.7337, -2.0168],
         [ 0.9555,  0.7594, -1.9725],
         [ 0.9570,  0.7893, -1.9274],
         ...,
         [ 1.1758,  1.4907,  1.7906],
         [ 1.1789,  1.4662,  1.8246],
         [ 1.1822,  1.4405,  1.8593]],

        ...,

        [[ 1.3530,  0.6545,  3.4689],
         [ 1.3512,  0.6912,  3.4029],
         [ 1.3490,  0.7340,  3.3395],
         ...,
         [ 0.9526,  2.3103, -1.0961],
         [ 0.9489,  2.2920, -1.1348],
         [ 0.9454,  2.2731, -1.1734]],

        [[

In [8]:
'''res_x = pandas.DataFrame(dtype=float) # append x 
res_y = pandas.DataFrame(dtype=float) # append y
res_z = pandas.DataFrame(dtype=float) #append z
leave = [] #append files which are not considered
for i in range(168):

    data = np.load(f'NMP/trimmed_Bamboo/trimmed_Bamboo_{i}.npz') #loading the file based on i value
    df = pandas.DataFrame.from_dict({item: data[item] for item in data.files}, orient='index') #converting into dataframe
    df.drop(['quaternion','frame_num', 'time_step'], inplace=True) #dropping unecessary columns
    #print(df)
    #concat
    info = df.T # Transpose the dataframe
    # Extract x, y, z positions
    info['x'] = info['position'].apply(lambda pos: pos[0])
    info['y'] = info['position'].apply(lambda pos: pos[1])
    info['z'] = info['position'].apply(lambda pos: pos[2])
    #info = info.drop(['position'], inplace=True, axis = 1)
    info.drop(['position'], inplace=True, axis=1)
    info = info.T
    # Transpose to keep it as a row
    x = pandas.DataFrame(info.loc['x'].iloc[:100]).T  
    y = pandas.DataFrame(info.loc['y'].iloc[:100]).T  
    z = pandas.DataFrame(info.loc['z'].iloc[:100]).T 
    
    if not df.shape[1] < 100: #if greater than 100 concat will happen
        info = info.iloc[:, :100]
        res_x = pandas.concat([res_x, x]) #default axis = 0
        res_y = pandas.concat([res_y, y]) #default axis = 0
        res_z = pandas.concat([res_z, z]) #default axis = 0
        print(f'NMP/trimmed_Bamboo/trimmed_Bamboo_{i}.npz')
        print(df)
    elif df.shape[1] < 100:
        leave.append(i)
    print("---------------------------")

#print('total files concatinated count: ', 168 - len(leave))
    '''

'res_x = pandas.DataFrame(dtype=float) # append x \nres_y = pandas.DataFrame(dtype=float) # append y\nres_z = pandas.DataFrame(dtype=float) #append z\nleave = [] #append files which are not considered\nfor i in range(168):\n\n    data = np.load(f\'NMP/trimmed_Bamboo/trimmed_Bamboo_{i}.npz\') #loading the file based on i value\n    df = pandas.DataFrame.from_dict({item: data[item] for item in data.files}, orient=\'index\') #converting into dataframe\n    df.drop([\'quaternion\',\'frame_num\', \'time_step\'], inplace=True) #dropping unecessary columns\n    #print(df)\n    #concat\n    info = df.T # Transpose the dataframe\n    # Extract x, y, z positions\n    info[\'x\'] = info[\'position\'].apply(lambda pos: pos[0])\n    info[\'y\'] = info[\'position\'].apply(lambda pos: pos[1])\n    info[\'z\'] = info[\'position\'].apply(lambda pos: pos[2])\n    #info = info.drop([\'position\'], inplace=True, axis = 1)\n    info.drop([\'position\'], inplace=True, axis=1)\n    info = info.T\n    # Trans

In [9]:
'''# may use for column wise input
def forward(self, x, future=0): #updated forwad
        outputs = []
        n_samples = x.size(0)  # Batch size
        seq_len = x.size(1)  # Number of time steps
        n_features = x.size(2)  # Number of features (should be 3 for x, y, z)

        h_t = torch.zeros(n_samples, self.n_hidden, dtype=torch.float32)
        c_t = torch.zeros(n_samples, self.n_hidden, dtype=torch.float32) 
        h_t2 = torch.zeros(n_samples, self.n_hidden, dtype=torch.float32) 
        c_t2 = torch.zeros(n_samples, self.n_hidden, dtype=torch.float32) 

        # Iterate over each time step
        for t in range(seq_len):
            input = x[:, t, :]  # Extract the t-th time step across all sequences; shape is [batch_size, 3]
            h_t, c_t = self.lstm1(input, (h_t, c_t))
            h_t2, c_t2 = self.lstm2(h_t, (h_t2, c_t2))
            output = self.linear(h_t2)  # output will have shape [batch_size, 3]
            outputs.append(output)

        # Predict future values if required
        for i in range(future):
            h_t, c_t = self.lstm1(output, (h_t, c_t))
            h_t2, c_t2 = self.lstm2(h_t, (h_t2, c_t2))
            output = self.linear(h_t2)
            outputs.append(output)

        outputs = torch.stack(outputs, dim=1)  # Stack the outputs along the sequence dimension
        return outputs'''

'# may use for column wise input\ndef forward(self, x, future=0): #updated forwad\n        outputs = []\n        n_samples = x.size(0)  # Batch size\n        seq_len = x.size(1)  # Number of time steps\n        n_features = x.size(2)  # Number of features (should be 3 for x, y, z)\n\n        h_t = torch.zeros(n_samples, self.n_hidden, dtype=torch.float32)\n        c_t = torch.zeros(n_samples, self.n_hidden, dtype=torch.float32) \n        h_t2 = torch.zeros(n_samples, self.n_hidden, dtype=torch.float32) \n        c_t2 = torch.zeros(n_samples, self.n_hidden, dtype=torch.float32) \n\n        # Iterate over each time step\n        for t in range(seq_len):\n            input = x[:, t, :]  # Extract the t-th time step across all sequences; shape is [batch_size, 3]\n            h_t, c_t = self.lstm1(input, (h_t, c_t))\n            h_t2, c_t2 = self.lstm2(h_t, (h_t2, c_t2))\n            output = self.linear(h_t2)  # output will have shape [batch_size, 3]\n            outputs.append(output)\n\n

In [30]:
class TransformerModel(nn.Module):
    def __init__(self, n_input=3, n_hidden=128, n_output=3, n_layers=3, n_heads=8, dropout=0.1):
        super(TransformerModel, self).__init__()
        self.n_hidden = n_hidden
        self.embedding = nn.Linear(n_input, n_hidden)
        self.positional_encoding = PositionalEncoding(n_hidden, dropout)
        
        # Transformer encoder
        encoder_layer = nn.TransformerEncoderLayer(d_model=n_hidden, nhead=n_heads)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=n_layers)
        
        # Decoder to map to output dimensions
        self.decoder = nn.Linear(n_hidden, n_output)

    # In the forward method of your Transformer model
    def forward(self, src, future=0):
        src = self.embedding(src) * torch.sqrt(torch.tensor(self.n_hidden, dtype=torch.float32))
        src = self.positional_encoding(src)
        
        # Permute to [sequence_length, batch_size, embedding_size] if necessary
        src = src.permute(1, 0, 2)  # Shape adjustment for transformer input
        
        # Pass through the transformer encoder
        output = self.transformer_encoder(src)
        
        # Permute back to original [batch_size, sequence_length, embedding_size]
        output = output.permute(1, 0, 2)
        output = self.decoder(output)
        
        return output


class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        
        # Compute the positional encodings
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-torch.log(torch.tensor(10000.0)) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

In [8]:
class lstmpred(nn.Module):
    def __init__(self, n_hidden=51):
        super(lstmpred, self).__init__()
        self.n_hidden = n_hidden
        self.lstm1 = nn.LSTMCell(3, self.n_hidden)
        self.lstm2 = nn.LSTMCell(self.n_hidden, self.n_hidden)
        self.lstm3 = nn.LSTMCell(self.n_hidden, self.n_hidden)  # lstm3
        self.lstm4 = nn.LSTMCell(self.n_hidden, self.n_hidden)  # lstm4
        self.lstm5 = nn.LSTMCell(self.n_hidden, self.n_hidden)  # lstm5 (new layer)
        self.lstm6 = nn.LSTMCell(self.n_hidden, self.n_hidden)  # lstm6 (new layer)
        self.linear = nn.Linear(self.n_hidden, 3)

    def forward(self, x, future=0):
        outputs = []
        n_samples = x.size(0)  # Batch size

        # Initialize hidden and cell states for all LSTM layers
        h_t = torch.zeros(n_samples, self.n_hidden, dtype=torch.float32).to(device)
        c_t = torch.zeros(n_samples, self.n_hidden, dtype=torch.float32).to(device)
        h_t2 = torch.zeros(n_samples, self.n_hidden, dtype=torch.float32).to(device)
        c_t2 = torch.zeros(n_samples, self.n_hidden, dtype=torch.float32).to(device)
        #h_t3 = torch.zeros(n_samples, self.n_hidden, dtype=torch.float32).to(device)
        #c_t3 = torch.zeros(n_samples, self.n_hidden, dtype=torch.float32).to(device)
        #h_t4 = torch.zeros(n_samples, self.n_hidden, dtype=torch.float32).to(device)
        #c_t4 = torch.zeros(n_samples, self.n_hidden, dtype=torch.float32).to(device)
        #h_t5 = torch.zeros(n_samples, self.n_hidden, dtype=torch.float32).to(device)  # For lstm5
        #c_t5 = torch.zeros(n_samples, self.n_hidden, dtype=torch.float32).to(device)  # For lstm5
        #h_t6 = torch.zeros(n_samples, self.n_hidden, dtype=torch.float32).to(device)  # For lstm6
        #c_t6 = torch.zeros(n_samples, self.n_hidden, dtype=torch.float32).to(device)  # For lstm6

        for input in x.split(1, dim=1):  # Input shape [batch_size, 1, 3]
            input = input.squeeze(1)  # Shape [batch_size, 3]
            h_t, c_t = self.lstm1(input, (h_t, c_t))
            h_t2, c_t2 = self.lstm2(h_t, (h_t2, c_t2))
            #h_t3, c_t3 = self.lstm3(h_t2, (h_t3, c_t3))
            #h_t4, c_t4 = self.lstm4(h_t3, (h_t4, c_t4))
            #h_t5, c_t5 = self.lstm5(h_t4, (h_t5, c_t5))  # Add lstm5 layer processing
            #h_t6, c_t6 = self.lstm6(h_t5, (h_t6, c_t6))  # Add lstm6 layer processing
            output = self.linear(h_t2)  # Output from the last LSTM layer (lstm6)
            outputs.append(output)

        for i in range(future):
            h_t, c_t = self.lstm1(output, (h_t, c_t))
            h_t2, c_t2 = self.lstm2(h_t, (h_t2, c_t2))
            #h_t3, c_t3 = self.lstm3(h_t2, (h_t3, c_t3))
            #h_t4, c_t4 = self.lstm4(h_t3, (h_t4, c_t4))
            #h_t5, c_t5 = self.lstm5(h_t4, (h_t5, c_t5))  # Add lstm5 layer processing
            #h_t6, c_t6 = self.lstm6(h_t5, (h_t6, c_t6))  # Add lstm6 layer processing
            output = self.linear(h_t2)
            outputs.append(output)

        outputs = torch.stack(outputs, dim=1)  # Shape [batch_size, sequence_length, 3]
        return outputs

In [9]:
train_input = combined_tensor[3:, :-1]   # leave the last time step for training input
train_target = combined_tensor[3:, 1:]   # training target is shifted by one step
test_input = combined_tensor[:3, :-1]    # first 3 sequences for testing
test_target = combined_tensor[:3, 1:]    # testing target is also shifted by one step

train_input = train_input.permute(1, 0, 2)  # Adjust for transformer input
train_target = train_target.permute(1, 0, 2)  # Ensure targets match shape as needed

In [29]:
###this is for  overall trajectory optimisation
# Initialize the model
model = lstmpred()  
model = model.to(device)

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.LBFGS(model.parameters(), lr=0.8)
n_steps = 100  # Number of steps to train the model

for i in range(n_steps):
    print("Step:", i + 1)

    def closure():
        optimizer.zero_grad()
        out = model(train_input)
        loss = criterion(out, train_target)
        print('Training loss:', loss.item())
        loss.backward()
        return loss

    optimizer.step(closure)
    pid = os.getpid()
    py = psutil.Process(pid)
    memoryUse = py.memory_info()[0] / 2.**30  # Convert from bytes to GB
    print(f"Memory Usage: {memoryUse:.2f} GB")

    # Testing phase
    with torch.no_grad():
        future = 100  # Predict 100 future steps beyond the test sequence
        pred = model(test_input, future=future)
        loss = criterion(pred[:, :-future], test_target)
        print('Test loss:', loss.item())
        y_pred = pred.cpu().detach().numpy()

Step: 1
Training loss: 2.9512009620666504
Training loss: 2.7762298583984375
Training loss: 0.728507936000824
Training loss: 0.6934286952018738
Training loss: 0.6912650465965271
Training loss: 1.2055586576461792
Training loss: 0.5616387724876404
Training loss: 0.44388115406036377
Training loss: 0.307002991437912
Training loss: 34.48451232910156
Training loss: 0.568917453289032
Training loss: 0.08866420388221741
Training loss: 0.0484614372253418
Training loss: 0.04196939617395401
Training loss: 0.03928491845726967
Training loss: 0.03548590838909149
Training loss: 0.03186909854412079
Training loss: 0.027263665571808815
Training loss: 0.02123185060918331
Training loss: 0.01555304229259491
Memory Usage: 1.80 GB
Test loss: 1.532914161682129
Step: 2
Training loss: 0.010161904618144035
Training loss: 0.006775463465601206
Training loss: 0.0053020017221570015
Training loss: 0.004439494106918573
Training loss: 0.0035180365666747093
Training loss: 0.002137480303645134
Training loss: 0.001497420249

In [27]:
torch.save(model.state_dict(), 'model-overall.pth')

In [14]:
'''# Model initialization
model = lstmpred().to(device)

# Loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.LBFGS(model.parameters(), lr=0.8)

# Training for 1100 sequences
n_steps = 30  # Number of optimization steps

for i in range(n_steps):
    print("Step:", i + 1)

    # Closure function for LBFGS optimizer
    def closure():
        optimizer.zero_grad()
        total_loss = 0.0

        for start in range(99):  # Iterate over possible starting points
            train_input = combined_tensor[:, :start + 1, :]  # Use start+1 columns as input
            train_target = combined_tensor[:, start + 1:, :]  # Predict the remaining columns

            # Check tensor sizes for debugging
            assert train_input.size(1) <= combined_tensor.size(1), f"Input size {train_input.size()} exceeds tensor limits"
            assert train_target.size(1) + start + 1 == combined_tensor.size(1), f"Target size {train_target.size()} mismatch"

            out = model(train_input, future=train_target.size(1))
            loss = criterion(out[:, -train_target.size(1):], train_target)
            total_loss += loss

        total_loss.backward()
        print(f'Training loss at step {i+1}: {total_loss.item()}')

        # Check memory consumption
        pid = os.getpid()
        py = psutil.Process(pid)
        memoryUse = py.memory_info()[0] / 2.**30  # Convert from bytes to GB
        print(f"Memory Usage: {memoryUse:.2f} GB")
        return total_loss

    # Optimizer step
    optimizer.step(closure)

    # Testing phase
    with torch.no_grad():
        future = 99  # Predict remaining steps for the last test sequence
        pred = model(test_input, future=future)
        loss = criterion(pred[:, -future:], test_target)
        print(f'Test loss at step {i+1}: {loss.item()}')

        # Check and print shape of the prediction for debugging
        print(f'Prediction shape: {pred.shape}')
        y_pred = pred.cpu().detach().numpy()
'''

'# Model initialization\nmodel = lstmpred().to(device)\n\n# Loss function and optimizer\ncriterion = nn.MSELoss()\noptimizer = optim.LBFGS(model.parameters(), lr=0.8)\n\n# Training for 1100 sequences\nn_steps = 30  # Number of optimization steps\n\nfor i in range(n_steps):\n    print("Step:", i + 1)\n\n    # Closure function for LBFGS optimizer\n    def closure():\n        optimizer.zero_grad()\n        total_loss = 0.0\n\n        for start in range(99):  # Iterate over possible starting points\n            train_input = combined_tensor[:, :start + 1, :]  # Use start+1 columns as input\n            train_target = combined_tensor[:, start + 1:, :]  # Predict the remaining columns\n\n            # Check tensor sizes for debugging\n            assert train_input.size(1) <= combined_tensor.size(1), f"Input size {train_input.size()} exceeds tensor limits"\n            assert train_target.size(1) + start + 1 == combined_tensor.size(1), f"Target size {train_target.size()} mismatch"\n\n       

In [15]:
'''
# Model, loss function, and optimizer initialization
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = TransformerModel().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
n_steps = 30

for i in range(n_steps):
    print("Step:", i + 1)
    model.train()
    total_loss = 0.0
    
    for start in range(99):
        optimizer.zero_grad()
        train_input = combined_tensor[:, :start + 1, :].to(device)
        train_target = combined_tensor[:, start + 1:, :].to(device)
        
        # Forward pass
        out = model(train_input, future=train_target.size(1))
        loss = criterion(out[:, -train_target.size(1):], train_target)
        
        # Backpropagation
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        
    print(f'Training loss at step {i+1}: {total_loss / 99}')
    
    # Testing phase
    model.eval()
    with torch.no_grad():
        test_input = test_input.to(device)
        test_target = test_target.to(device)
        
        future = 99
        pred = model(test_input, future=future)
        test_loss = criterion(pred[:, -future:], test_target)
        print(f'Test loss at step {i+1}: {test_loss.item()}')
        print(f'Prediction shape: {pred.shape}')'''

Step: 1


c:\Users\Mahesh Reddy\anaconda3\envs\dl\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([1150, 99, 3])) that is different to the input size (torch.Size([1150, 1, 3])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
c:\Users\Mahesh Reddy\anaconda3\envs\dl\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([1150, 98, 3])) that is different to the input size (torch.Size([1150, 2, 3])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


RuntimeError: The size of tensor a (2) must match the size of tensor b (98) at non-singleton dimension 1

In [30]:
position_data[:,1]

array([1.195604, 1.229114, 1.263349, 1.295981, 1.328077, 1.359165,
       1.389401, 1.418738, 1.44709 , 1.475127, 1.500655, 1.527649,
       1.552774, 1.578319, 1.602128, 1.625138, 1.647395, 1.669222,
       1.690642, 1.711912, 1.731866, 1.751615, 1.770708, 1.789455,
       1.807435, 1.824953, 1.842787, 1.859558, 1.875816, 1.891673,
       1.906938, 1.921628, 1.935879, 1.949543, 1.962566, 1.974939,
       1.98657 , 1.997395, 2.007589, 2.016975, 2.025596, 2.033045,
       2.040154, 2.046382, 2.051799, 2.056284, 2.059945, 2.062873,
       2.065034, 2.066409, 2.067163, 2.0672  , 2.066064, 2.064118,
       2.061924, 2.058638, 2.054467, 2.049568, 2.044126, 2.038397,
       2.031304, 2.023955, 2.015725, 2.006957, 1.997467, 1.987215,
       1.976334, 1.964731, 1.952329, 1.939871, 1.92645 , 1.912514,
       1.898089, 1.882936, 1.866732, 1.850798, 1.83392 , 1.815987,
       1.79727 , 1.779388, 1.761171, 1.741985, 1.722183, 1.701611,
       1.680734, 1.659763, 1.637187, 1.615453, 1.592198, 1.568

In [31]:
test_target_4.size(1)

50

In [32]:

model.eval()
with torch.no_grad():
    # Predict the second half of the sequence based on the first half
    future_steps = test_target_4.size(1)  # Number of future steps equals the length of the second half
    prediction = model(test_input_4, future=future_steps)

    # Extract the predicted second half
    predicted_output = prediction[:, -future_steps:]  # Only take the predicted future steps

    # Move the tensors to CPU and convert to NumPy arrays
    prediction = prediction.cpu().detach().numpy()
    predicted_output = predicted_output.cpu().detach().numpy()

In [33]:
type(position_data)

numpy.ndarray

In [19]:
position_data = position_data.cpu().numpy()

In [34]:
prediction.squeeze(0)

array([[ 0.9451176 ,  0.911494  ,  2.7955816 ],
       [ 0.8162242 ,  0.34838986,  4.6546345 ],
       [ 0.758367  , -0.01734436,  5.5834427 ],
       [ 0.7332287 , -0.18909574,  6.014058  ],
       [ 0.72092783, -0.23300755,  6.1848135 ],
       [ 0.7131781 , -0.19992268,  6.2176228 ],
       [ 0.7068435 , -0.12260568,  6.1760592 ],
       [ 0.7008642 , -0.0213629 ,  6.0945797 ],
       [ 0.69497675,  0.09217429,  5.9917192 ],
       [ 0.68919176,  0.21114051,  5.877921  ],
       [ 0.6836208 ,  0.330882  ,  5.7597117 ],
       [ 0.6783553 ,  0.45032358,  5.639267  ],
       [ 0.6734742 ,  0.56796974,  5.518634  ],
       [ 0.6690439 ,  0.68353   ,  5.3984146 ],
       [ 0.6651195 ,  0.795996  ,  5.279756  ],
       [ 0.66175556,  0.90508157,  5.163019  ],
       [ 0.6589904 ,  1.0108855 ,  5.0481205 ],
       [ 0.65685034,  1.1136553 ,  4.9347844 ],
       [ 0.65535426,  1.2135192 ,  4.8228483 ],
       [ 0.6545063 ,  1.3102015 ,  4.712611  ],
       [ 0.6543351 ,  1.4041381 ,  4.603

In [35]:
prediction = np.squeeze(prediction, axis=0)

In [36]:
prediction.shape

(100, 3)

In [37]:
# Extract x, y, z coordinates from the predictions
predicted_x = prediction[:, 0]
predicted_y = prediction[:, 1]
predicted_z = prediction[:, 2]


actual_x = position_data[:, 0]
actual_y = position_data[:, 1]
actual_z = position_data[:, 2]

# Create the 3D line plot for the actual data (first half)
trace_actual = go.Scatter3d(
    x=actual_x,
    y=actual_y,
    z=actual_z,
    mode='lines+markers',  # Both lines and markers
    marker=dict(size=5, color='blue'),  # Marker size and color for actual data
    line=dict(color='blue', width=2),  # Line color and width for actual data
    name='Actual Data (First Half)'
)

# Create the 3D line plot for the predicted data (second half)
trace_predicted = go.Scatter3d(
    x=predicted_x,
    y=predicted_y,
    z=predicted_z,
    mode='lines+markers',  # Both lines and markers
    marker=dict(size=5, color='red'),  # Marker size and color for predicted data
    line=dict(color='red', width=2),  # Line color and width for predicted data
    name='Predicted Data (Second Half)'
)

# Create layout
layout = go.Layout(
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z'
    ),
    title="Interactive 3D Trajectory: Actual (First Half) and Predicted (Second Half)"
)

# Create the figure with both traces and display it
fig = go.Figure(data=[trace_actual, trace_predicted], layout=layout)
fig.show()